In [1]:
from neo4j import GraphDatabase
import pandas as pd

In [2]:
# "First Create an empty database, within a project"

uri = "bolt://localhost:7687"
username = "neo4j" # username for Neo4j, most probably is Neo4j, if you did not change while installing Neo4j Desktop
password = "eigen1234" # Password for database.
database_name = "d4.cora"  # Database name

driver = GraphDatabase.driver(uri, auth=(username, password))

In [3]:
def check_connection():
    try:
        # Establish a session with the specified database
        with driver.session() as session:
            # Run a simple query to check the connection
            result = session.run("RETURN 'Connection to database successful' AS message")
            for record in result:
                print(record["message"])
    except Exception as e:
        print("Error connecting to the database:", e)

# Call the check_connection function
check_connection()

Connection to database successful


In [4]:
# Function to create nodes in the specified database with a dynamic label
def create_nodes(data, driver, label):
    """Create nodes in Neo4j from a CSV file."""
    try:
        # Load the CSV data
        node_data = pd.read_csv(data)
        
        with driver.session() as session:
            for _, row in node_data.iterrows():
                properties = row.to_dict()
                properties['features'] = eval(properties['features'])  # Convert string back to list if saved as string
                query = f"CREATE (n:{label} {{id: $id, features: $features, label: $label}})"
                session.run(query, **properties)

    except Exception as e:
        print("Error during node creation:", e)
    finally:
        driver.close()

In [5]:
create_nodes("nodes.csv",driver,"cora")

In [6]:
def create_edges(data, driver, label):
    """Create undirected edges in Neo4j from a CSV file, ensuring only one edge is created between two nodes."""
    try:
        # Load the edge list
        edge_data = pd.read_csv(data)
        
        with driver.session() as session:
            for _, row in edge_data.iterrows():
                # Ensure the smaller id is always the 'source' and the larger id is the 'target'
                source_id = min(row['source_id'], row['target_id'])
                target_id = max(row['source_id'], row['target_id'])
                
                # Cypher query to create an undirected edge (just one edge, not two directions)
                query = f"""
                MATCH (source:{label} {{id: $source_id}})
                MATCH (target:{label} {{id: $target_id}})
                MERGE (source)-[:CITES_CORA {{value: 1}}]->(target)
                """
                
                # Run the query with the sorted source and target ids
                session.run(query, {"source_id": source_id, "target_id": target_id})

    except Exception as e:
        print("Error during edge creation:", e)
    finally:
        driver.close()


In [7]:
create_edges("edges.csv",driver,"cora")

/var/folders/5m/3t1ykjr55_555tllby0d28j00000gn/T/ipykernel_21890/33394405.py:7: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
